In [16]:
import json
import re

class TextoJson:
    def __init__(self, archivo):
        self.archivo = archivo
        self.data = {
            "informacion_paciente": {},
            "informacion_general": {},
            "diagnostico": {},
            "antecedentes_familiares": {},
            "habitos_estilo_vida": {},
            "historial_medico": {},
            "historia_personal": {},
            "comorbilidades": []
        }

        self.secciones = {
            "informacion_paciente": ["clave", "id", "edad", "medico_tratante"],
            "informacion_general": ["origen", "residencia", "ocupacion", "ss"],
            "habitos_estilo_vida": [],
            "historial_medico": [],
            "historia_personal": [],
            "comorbilidades": []
        }

    def preprocesar_texto(self):
        with open(self.archivo, 'r', encoding='utf-8') as file:
            lineas = file.readlines()

        texto_procesado = []
        clave_actual = None
        valor_actual = []

        for linea in lineas:
            linea = linea.strip()  # Eliminar espacios innecesarios
            if not linea:
                continue  # Ignorar líneas vacías

            # Identificar si la línea es una nueva clave
            if re.match(r'^[a-záéíóúñ]+.*?:', linea, re.IGNORECASE):
                # Si ya hay una clave en curso, guardar la clave y su valor
                if clave_actual:
                    texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")
                
                # Actualizar clave y reiniciar el valor
                clave_actual = linea.split(':', 1)[0].strip()
                valor_actual = [linea.split(':', 1)[1].strip()]
            elif clave_actual:
                # Agregar la línea al valor actual
                valor_actual.append(linea)
            else:
                ##este caso es cuando la clave no tiene dos puntos
                texto_procesado.append(linea)

        # Guardar la última clave-valor si quedó algo pendiente
        if clave_actual:
            texto_procesado.append(f"{clave_actual}: {' '.join(valor_actual).strip()}")

        print('\n'.join(texto_procesado))
        return '\n'.join(texto_procesado)

    def procesar(self):
        contenido = self.preprocesar_texto().split('\n')
        seccion_actual = None

        # Procesar las líneas del contenido
        for linea in contenido:
            linea = linea.strip().lower()
            
            # Capturar información personal
            match_id = re.match(r'^(gacmc)\s*/\s*(.+?)\s*/\s*(\d+\s*años)\s*/\s*(.*)$', linea)
            if match_id:
                self.data["informacion_paciente"]["clave"] = match_id.group(1).strip()
                self.data["informacion_paciente"]["id"] = match_id.group(2).strip()
                self.data["informacion_paciente"]["edad"] = match_id.group(3).strip()
                self.data["informacion_paciente"]["medico_tratante"] = match_id.group(4).strip()
                continue

             # Capturar información general
            origen_flag = False
            residencia_flag = False

            if "originaria" in linea or "origen" in linea:
                origen_flag = True
            if "residente" in linea or "residencia" in linea:
                residencia_flag = True

            if origen_flag and residencia_flag:  # Caso "origen y residencia"
                match = re.search(r':\s*(.+?)\s+y\s+(.+)', linea)
                if match:
                    self.data["informacion_general"]["origen"] = match.group(1).strip()
                    self.data["informacion_general"]["residencia"] = match.group(2).strip()
            elif origen_flag:  # Solo origen
                match = re.search(r'(originaria|origen):?\s*(.+)', linea)
                if match:
                    self.data["informacion_general"]["origen"] = match.group(2).strip()
            elif residencia_flag:  # Solo residencia
                match = re.search(r'(residente|residencia):?\s*(.+)', linea)
                if match:
                    self.data["informacion_general"]["residencia"] = match.group(2).strip()

            # Ocupación
            match_ocupacion = re.search(r'ocupación:?\s*(.+)', linea)
            if match_ocupacion:
                self.data["informacion_general"]["ocupacion"] = match_ocupacion.group(1).strip()

            # Seguridad social
            match_ss = re.search(r'^(seguridad social|ss):?\s*(.+)$', linea)  # Asegurar que la línea completa coincida
            if match_ss:
                self.data["informacion_general"]["ss"] = match_ss.group(2).strip()

            # diagnostico
            match_diagnostico = re.search(r'^(diagnostico|diagnóstico):?\s*(.+)$', linea)  # Asegurar que la línea completa coincida
            if match_diagnostico:
                self.data["diagnostico"] = match_diagnostico.group(2).strip()

            #antecedentes, cualquier tipo de antecedente como ahf, apf, etc
            match_antecedentes = re.search(r'^(ahf|apf|apnp|app|appnp|apm|apnm|appm|appnm|apmnp|apmnm|appmnp|appmnm|appmnmnp):?\s*(.+)$', linea)  # Asegurar que la línea completa coincida
            if match_antecedentes:
                self.data["antecedentes_familiares"][match_antecedentes.group(1)] = match_antecedentes.group(2).strip()


        self.imprimir()

            
    def guardar_json(self, salida):
        with open(salida, 'w', encoding='utf-8') as file:
            json.dump(self.data, file, indent=4, ensure_ascii=False)

    def imprimir(self):
        print(json.dumps(self.data, indent=4, ensure_ascii=False))

# Ejemplo de uso
archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"  # Archivo de entrada con el texto
archivo_salida = "resultado.json"  # Archivo JSON de salida

procesador = TextoJson(archivo_entrada)
procesador.procesar()
# procesador.guardar_json(archivo_salida)

GACMC / 84959 / 67 años / Dra. Martínez
Originaria y residente: Guanajuato y Ciudad de México
Ocupación: Directora de planeación CONALEP
Seguridad social: ISSSTE
Diagnóstico: CDI de mama derecha EC lA triple negativo
AHF oncológicos: Tía materna falleció por cáncer de mama / Prima materna con cáncer de mama (desconoce edad al diagnóstico)
Tabaco: negado
Alcohol: ocasional
Drogas: negado - Menarca 15 años / FUM 21 años -GOPOCOAO - TRH: durante 30 años - Estado hormonal: postmenopáusica
Cirugías: Esplenectomía abierta (1976) / Tiroidectomía (1988) / Pancreatectomia distal + Resección de implante gástrico + biopsia en cuña de segmento ll hepático + colecistectomía y maniobra de Kocher extendida (2016) / CC + GC (2017)
Comorbilidades: 1. Linfoma de Hodgkin (1975): 3 ciclos QT (ciclofosfamida, vincristina y otro fármaco) + RT por encima del diafragma (IMSS) 2. Cáncer papilar de tiroides (1988): Tiroidectomía + lodo 131 T4/T3 3. Cistoadenoma seroso microquístico de páncreas (2016) 4, CDI de 

<>:135: SyntaxWarning: invalid escape sequence '\D'
<>:135: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JOSUE\AppData\Local\Temp\ipykernel_9364\3950756853.py:135: SyntaxWarning: invalid escape sequence '\D'
  archivo_entrada = "D:\DOCUMENTOS\VirtualEnvPy\dataScience\source\Servicio\OCR_erik\TextoExtraido2.txt"  # Archivo de entrada con el texto
